In [1]:
#mount drive and fetch API token from secrets.
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive')
token = userdata.get('hubSpotAPI')

Mounted at /content/drive


In [2]:
#Sets up the function for API call and fetching data and storing in dictionary.
import requests
import pandas as pd
import openpyxl

headers = {
    'Authorization': f'Bearer {token}',
    'Content-Type': 'application/json'
}

def get_properties(object_type):
    url = f"https://api.hubapi.com/crm/v3/properties/{object_type}"
    response = requests.get(url, headers=headers)
    properties = response.json().get('results', [])
    return {prop['name']: prop for prop in properties}

In [ ]:
#Create separate excel file for each object.
objects = ['company', 'contact', 'deal', 'ticket', 'events', 'partnerships', 'call', 'feedback_submission', 'marketing_event', 'meeting', 'plans', 'product', 'quote', 'ticket', 'referrals', 'user', 'automation_platform_flow', 'line_item', 'tasks']

for object_type in objects:
    properties = get_properties(object_type)
    df = pd.DataFrame.from_dict(properties, orient='index')
    file_path = '/content/drive/My Drive/Metadata/' + object_type + '.xlsx'
    df.to_excel(file_path, index=False)

In [3]:
#Create one excel file and put all the data in one sheet by adding one column named object type.
objects = ['company', 'contact', 'deal', 'ticket', 'events', 'partnerships', 'call', 'feedback_submission', 'marketing_event', 'meeting', 'plans', 'product', 'quote', 'ticket', 'referrals', 'user', 'automation_platform_flow', 'line_item', 'tasks']
dataframes = []
for object_type in objects:
    properties = get_properties(object_type)
    df = pd.DataFrame.from_dict(properties, orient='index')
    df['type'] = object_type
    dataframes.append(df)
all_data = pd.concat(dataframes, ignore_index=True)
file_path = '/content/drive/My Drive/Metadata/all_objects_metadata.xlsx'
all_data.to_excel(file_path, index=False)

In [ ]:
#Create one excel file and put all the data into different sheets with the sheet name taken from object type.
master_file_path = '/content/drive/My Drive/Metadata/CRM_Metadata.xlsx'
with pd.ExcelWriter(master_file_path, engine='openpyxl') as writer:
    for object_type in objects:
        properties = get_properties(object_type)
        df = pd.DataFrame.from_dict(properties, orient='index')
        df.to_excel(writer, sheet_name=object_type, index=False)
